In [19]:
# import project directory helper
import os, sys, inspect
ROOT_DIR = os.path.abspath('/home/hduser/document/jupyter/FYP/') 
sys.path.insert(0, ROOT_DIR)

In [174]:
# import util packages
from IPython.display import display
import ipywidgets as w
import pandas as pd
import re

In [26]:
# import pyspark packages
# set the kafka dependencies before create spark context or session
import os
os.environ[
    'PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4,org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4 pyspark-shell'
from pyspark.sql import SparkSession, functions, types
from pyspark.sql.types import *

In [27]:
spark = SparkSession.builder.appName('attraction').getOrCreate()
spark

In [29]:
# define path
tripad_attr_path = ROOT_DIR + '/crawler/datasets/tripadvisor_dataset/attractions/'
parquet_path = ROOT_DIR + '/crawler/datasets/tripadvisor_dataset/spark/'

In [55]:
# read spark dataframe from parquet
final_attr_spark_df = spark.read.parquet(parquet_path + 'etl/attractions')

In [56]:
final_attr_spark_df.createOrReplaceTempView('final_attr_spark_df')

In [57]:
final_attr_spark_df.price

Column<b'price'>

In [58]:
attr_price_spark_df = spark.sql(
    "SELECT MIN(price) as min_price, MAX(price) as max_price FROM final_attr_spark_df"
)

In [63]:
attr_price_df = attr_price_spark_df.toPandas()

In [64]:
attr_category_df = final_attr_spark_df.select(final_attr_spark_df.category).distinct().toPandas()

# User details and Preferences

In [204]:
w_uname = w.Text(placeholder='Liho', description="User Name")
w_destination = w.Text(placeholder='Kuala Lumpur', description="Destination")
w_budget = w.IntRangeSlider(
    min=attr_price_df.min_price[0],
    max=attr_price_df.max_price[0],
    step=10,
    value=[attr_price_df.min_price[0], attr_price_df.max_price[0]],
    description="Budget")
v1 = w.VBox([uname, place])

w_start_date = w.DatePicker(description='Start Date', disabled=False)
w_end_date = w.DatePicker(description='End Date', disabled=False)
v2 = w.VBox([start, end])

out = w.HBox([v1, v2, budget])
display(out)

In [205]:
attr_category_list = attr_category_df.category.tolist()

In [206]:
attr_category_title_list = [' '.join(i.split('_')).title() for i in attr_category_list]

In [211]:
w_cat_rating = dict()


def btn_cat_rating(btn):
    cat = '_'.join(btn.description.split(' ')).lower()

    if cat in w_cat_rating:
        return
    print(btn.description)
    slider = w.IntSlider(min=0, max=5, step=1, description='Rate')
    display(slider)
    w_cat_rating[cat] = slider
    if (len(w_cat_rating) < 5):
        print("Rate {x} more!\n".format(x=5 - len(w_cat_rating)))

In [212]:
button_layout = w.Layout(width='100%', height='50px', border="1px solid black")
button_items = [
    w.Button(description=cat, layout=button_layout)
    for cat in attr_category_title_list
]

on_click = [item.on_click(btn_cat_rating) for item in button_items]

buttons = w.GridBox(button_items,
                    layout=w.Layout(grid_template_columns="repeat(4, 24%)"))

print("Select and rate at least 5 categories and rate them: ")
display(buttons)

Select and rate at least 5 categories and rate them: 


GridBox(children=(Button(description='Air Helicopter Balloon Tours', layout=Layout(border='1px solid black', h…

Cruises Sailing Water Tours


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

Shows Concerts Sports


IntSlider(value=0, description='Rate', max=5)

Rate 3 more!

Weddings Honeymoons


IntSlider(value=0, description='Rate', max=5)

Rate 2 more!



In [229]:
uname = re.sub(' ', '_', w_uname.value.lower())
destination = re.sub(' ', '_', w_destination.value.lower())

(budget_low, budget_high) = tuple([float(i) for i in w_budget.value])

start_date = w_start_date.value
end_date = w_end_date.value
    
cat_rating = {key: float(value.value) for key, value in w_cat_rating.items()}